In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from functools import partial
from random import choices, randint, randrange, random, sample
from typing import List, Optional, Callable, Tuple
import numpy as np
# from geneticalgorithm import geneticalgorithm as ga
import pandas as pd
from collections import Counter
from tqdm import tqdm
import time
from Bio.SeqUtils import MeltingTemp
from Bio import SeqIO
from plotly import graph_objects as go
import json
from imp import reload
import primer_selection
reload(primer_selection)
# import testing
# reload(testing)
import math
import Thermo
reload(Thermo)

/tmp/ipykernel_3796973/2147551372.py:18: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload


<module 'Thermo' from '/mnt/storage10/lwang/Projects/Amplicon_design_tool/amplicon4tb/Thermo.py'>

In [6]:
pd.read_csv('/mnt/storage10/lwang/Projects/Amplicon_design_tool/test/Amplicon_design_output/Primer_design-accepted_primers-42-400.csv').head().to_excel('test.xlsx')

In [7]:
genes = pd.read_csv('/mnt/storage10/lwang/Projects/Amplicon_design_tool/db/MTB-h37rv_asm19595v2-eg18.gff', sep = '\t', header = None, skiprows=[0,1,2,3,4,5,6])
genes = genes[genes[2]=='gene']
genes['Name'] = genes[8].str.extract(r'Name=([^;]*)')
genes = genes[[3,4,'Name']]
genes.columns = ['start', 'end', 'gene']

In [9]:
genes = genes[~genes['gene'].isna()]

In [7]:
df = pd.read_csv('/mnt/storage10/lwang/Projects/Amplicon_design_tool/db/variants.csv')
# df = df[~df['drugs'].isna()]
# df = df[~df['type'].isin(['synonymous_variant','non_coding_transcript_exon_variant'])]
# df = df[df['gene']=='fabG1']

# print(df['genome_pos'].min())
# print(df['genome_pos'].max())

In [10]:
df1['gene'].value_counts()/df['gene'].value_counts()

gene
PPE35      NaN
Rv1258c    NaN
Rv1979c    NaN
Rv2752c    NaN
Rv3083     NaN
Rv3236c    NaN
aftB       NaN
ahpC       NaN
ald        NaN
alr        NaN
atpE       NaN
ccsA       NaN
clpC1      NaN
ddn        NaN
eis        NaN
embA       NaN
embB       NaN
embC       NaN
embR       NaN
ethA       NaN
ethR       NaN
fabG1      1.0
fbiA       NaN
fbiB       NaN
fbiC       NaN
fbiD       NaN
fgd1       NaN
folC       NaN
fprA       NaN
gid        NaN
gyrA       NaN
gyrB       NaN
inhA       NaN
kasA       NaN
katG       NaN
mmpL5      NaN
mmpR5      NaN
mmpS5      NaN
mshA       NaN
ndh        NaN
panD       NaN
pepQ       NaN
pncA       NaN
ribD       NaN
rplC       NaN
rpoA       NaN
rpoB       NaN
rpoC       NaN
rpsA       NaN
rpsL       NaN
rrl        NaN
rrs        NaN
thyA       NaN
thyX       NaN
tlyA       NaN
ubiA       NaN
whiB6      NaN
whiB7      NaN
Name: count, dtype: float64

In [4]:
df = pd.read_csv('/mnt/storage10/lwang/Projects/Amplicon_design_tool/db/variants.csv')
df1 = df[df['gene']=='fabG1']
df1['change'].unique()

array(['c.-15C>T', 'c.-60C>G', 'c.-8T>C', ..., 'p.Glu59Gly', 'p.Ala43Pro',
       'p.Arg25Pro'], dtype=object)

In [17]:
import pandas as pd

# Example Series
series1 = pd.Series({'fabG1': 3, 'PPE35': 1, 'fprA': 1}, name='count')
series2 = pd.Series({'fabG1': 10, 'PPE35': 10, 'fprA': 10, 'gid': 10}, name='count')

series1_aligned = series1.reindex(series2.index).fillna(0)

# Calculate the ratio and format as string
result = series1_aligned / series2
formatted_result = result.apply(lambda x: f"{x:.0%}")

# Combine the two series
combined_series = series1_aligned.astype(str) + "/" + series2.astype(str) + " (" + formatted_result + ")"

print(combined_series)


fabG1    3.0/10 (30%)
PPE35    1.0/10 (10%)
fprA     1.0/10 (10%)
gid       0.0/10 (0%)
Name: count, dtype: object


In [11]:
df['gene'].value_counts()

gene
gyrA       670971
rrs        522322
rpoC       508252
rrl        503108
embA       289453
mmpL5      285863
rpoB       278048
clpC1      246191
embB       235966
rpsL       189191
PPE35      188896
fprA       186174
gid        172437
Rv1979c    170503
whiB6      169146
katG       165235
ald        164828
embC       161971
rpsA       161781
tlyA       149695
fgd1       135426
rpoA       129223
aftB       127931
mshA       126670
fbiC       117931
kasA       106268
gyrB        77837
Rv2752c     75833
pncA        74039
ccsA        73707
inhA        66439
Rv3236c     63067
thyA        61490
Rv3083      56273
ndh         53898
Rv1258c     50703
ethA        50536
ahpC        47889
rplC        43808
mmpS5       43471
folC        39463
thyX        37946
ubiA        36804
fbiB        35925
fabG1       33327
fbiD        32937
alr         30611
eis         27123
pepQ        25431
embR        21930
atpE        19244
whiB7       13313
panD        13007
ethR        11853
fbiA        10812
ribD 

In [12]:
df1['gene'].value_counts()

gene
fabG1    33327
Name: count, dtype: int64

In [5]:
reference_design = './test/Amplicon_design_output/Primer_design-accepted_primers-10-400.csv'
pd.read_csv(reference_design, header = 0)

,pLeft_ID,pLeft_coord,pLeft_length,pLeft_Tm,pLeft_GC,pLeft_Sequences,pLeft_EndStability,pRight_ID,pRight_coord,pRight_length,pRight_Tm,pRight_GC,pRight_Sequences,pRight_EndStability,Penalty,Product_size,Amplicone_type,Redesign,Designed_ranges
0,P1-L0,761087,20,60.033405,50.000000,NaN,2.57,P1-R0,761445,18,60.726180,66.666667,NaN,3.69,2.759585,358,Non_specific,['-'],"[761087, 761445]"
1,P2-L0,2155074,20,59.827648,60.000000,NaN,3.51,P2-R0,2155436,20,60.179081,60.000000,NaN,4.11,0.351433,362,Non_specific,['-'],"[2155074, 2155436]"
2,P3-L0,4247449,20,59.822859,55.000000,NaN,2.24,P3-R0,4247838,19,60.004320,52.631579,NaN,2.85,1.181461,389,Non_specific,['-'],"[4247449, 4247838]"
3,P4-L0,2288903,20,60.179194,60.000000,NaN,3.46,P4-R0,2289254,20,60.599204,60.000000,NaN,6.13,0.778398,351,Non_specific,['-'],"[2288903, 2289254]"
4,P5-L0,781654,19,60.446445,57.894737,NaN,2.90,P5-R0,782004,20,60.108992,55.000000,NaN,4.02,1.555437,350,Non_specific,['-'],"[781654, 782004]"
5,P6-L1,4247250,20,60.750911,60.000000,NaN,4.30,P6-R1,4247602,18,60.437359,66.666667,NaN,5.19,3.188270,352,Non_specific,['-'],"[4247250, 4247602]"
6,P7-L0,1673419,20,59.970965,60.000000,NaN,5.14,P7-R0,1673835,20,59.905954,50.000000,NaN,4.06,0.123080,416,Non_specific,['-'],"[1673419, 1673835]"
7,P8-L0,7344,20,60.109458,55.000000,NaN,4.04,P8-R0,7698,20,60.179081,60.000000,NaN,2.92,0.288539,354,Non_specific,['-'],"[7344, 7698]"
8,P9-L0,2288582,19,60.599797,63.157895,NaN,4.45,P9-R0,2288998,20,60.108615,60.000000,NaN,4.34,1.708412,416,Non_specific,['-'],"[2288582, 2288998]"
9,P10-L0,1472411,20,60.038526,55.000000,NaN,4.35,P10-R0,1472826,20,60.037605,55.000000,NaN,6.01,0.076131,415,Non_specific,['-'],"[1472411, 1472826]"


In [5]:
susana_design = pd.read_excel('/mnt/storage10/lwang/Projects/Amplicone_design_tool/model2in1/amplicon_TB_2023_V2.xlsx', header = 0)
susana_design = susana_design.iloc[:-3]

In [6]:
susana_design = pd.read_excel('/mnt/storage10/lwang/Projects/Amplicone_design_tool/model2in1/amplicon_TB_2023_V2.xlsx', header = 0)
susana_design = susana_design.iloc[:-3]
susana_design.to_csv('reference_design.csv', index = False)

In [16]:
df1 = pd.read_csv('/mnt/storage10/lwang/Projects/Amplicone_design_tool/model2in1/test/Amplicon_design_output/Primer_design-accepted_primers-4-400.csv', header = 0)

In [17]:
def modify_primer_name(primer, amplicone_type):
    prefix = 'sp' if 'Gene_specific' in amplicone_type elif 'ns' if 'Non_specific' in amplicone_type elif 'spol' if 'spol' in amplicone_type else ''
    parts = primer.split('-')
    if len(parts) > 1:
        parts[-1] = 'L' + str(int(parts[-1][1:]) + 1)  # Increment the number
    parts.insert(1, prefix)
    return '-'.join(parts)

# Apply modifications
df1['pLeft_ID'] = df1.apply(lambda x: modify_primer_name(x['pLeft_ID'], x['Amplicone_type']), axis=1)
df1['pRight_ID'] = df1.apply(lambda x: modify_primer_name(x['pRight_ID'], x['Amplicone_type']), axis=1)

In [18]:
df1

,index,pLeft_ID,pLeft_coord,pLeft_length,pLeft_Tm,pLeft_GC,pLeft_Sequences,pLeft_EndStability,pRight_ID,pRight_coord,pRight_length,pRight_Tm,pRight_GC,pRight_Sequences,pRight_EndStability,Penalty,Product_size,Amplicone_type,Redesign,Designed_ranges
0,0,P1-sp-L1,761087,20,60.033405,50.0,AGCCAGCTGAGCCAATTCAT,2.57,P1-sp-L1,761445,18,60.726180,66.666667,CGATCGATGCGGACGGTC,3.69,2.759585,358,Gene_specific,['-'],"[761087, 761445]"
1,0,P2-sp-L1,2155074,20,59.827648,60.0,CTCTTCGTCAGCTCCCACTC,3.51,P2-sp-L1,2155436,20,60.179081,60.000000,GGGTTCACGTAGATCAGCCC,4.11,0.351433,362,Gene_specific,['-'],"[2155074, 2155436]"
2,0,P1-ns-L1,4247449,20,59.822859,55.0,GCCGGCTACATGTCCAACTA,2.24,P1-ns-L1,4247838,19,60.004320,52.631579,TTACCGCCGRATTCACACT,2.85,1.181461,389,Non_specific,['-'],"[4247449, 4247838]"
3,0,P2-ns-L1,2288903,20,60.179194,60.0,CCGTTCTCGTCGACTCCTTC,3.46,P2-ns-L1,2289254,20,60.599204,60.000000,CCCGCATACGTCCACCATAC,6.13,0.778398,351,Non_specific,['-'],"[2288903, 2289254]"


In [9]:
df1

,index,pLeft_ID,pLeft_coord,pLeft_length,pLeft_Tm,pLeft_GC,pLeft_Sequences,pLeft_EndStability,pRight_ID,pRight_coord,pRight_length,pRight_Tm,pRight_GC,pRight_Sequences,pRight_EndStability,Penalty,Product_size,Amplicone_type,Redesign,Designed_ranges
0,0,P1-L0,761087,20,60.033405,50.0,AGCCAGCTGAGCCAATTCAT,2.57,P1-R0,761445,18,60.726180,66.666667,CGATCGATGCGGACGGTC,3.69,2.759585,358,Gene_specific,['-'],"[761087, 761445]"
1,0,P2-L0,2155074,20,59.827648,60.0,CTCTTCGTCAGCTCCCACTC,3.51,P2-R0,2155436,20,60.179081,60.000000,GGGTTCACGTAGATCAGCCC,4.11,0.351433,362,Gene_specific,['-'],"[2155074, 2155436]"
2,0,P1-L0,4247449,20,59.822859,55.0,GCCGGCTACATGTCCAACTA,2.24,P1-R0,4247838,19,60.004320,52.631579,TTACCGCCGRATTCACACT,2.85,1.181461,389,Non_specific,['-'],"[4247449, 4247838]"
3,0,P2-L0,2288903,20,60.179194,60.0,CCGTTCTCGTCGACTCCTTC,3.46,P2-R0,2289254,20,60.599204,60.000000,CCCGCATACGTCCACCATAC,6.13,0.778398,351,Non_specific,['-'],"[2288903, 2289254]"


In [40]:
seq = 'AGAGTTGGTGCGGCGTAA'
(dg, tm) = Thermo.simplified_tm(seq, complement_sequence(seq), TmVar)


{'Gibbs': -24.099999999999998, 'Entropy': -375.6, 'Enthalpy': -140.6, 'Gibbs5P': -5.74, 'Gibbs3P': -7.260000000000001}
<class 'dict'>
Oligo: AGAGTTGGTGCGGCGTAA
ThermoDict: {'Gibbs': -24.099999999999998, 'Entropy': -375.6, 'Enthalpy': -140.6, 'Gibbs5P': -5.74, 'Gibbs3P': -7.260000000000001}
TmVar: <class '__main__.TmVar'>
Type of ThermoDict: <class 'dict'>


NameError: name 'pd' is not defined

In [48]:
ref_genome = 'MTB-h37rv_asm19595v2-eg18.fa'

def find_sequence_location(query_seq, fasta_file=ref_genome):
    ref_genome = SeqIO.read(fasta_file, "fasta")
    position = ref_genome.seq.find(query_seq)
    if position != -1:
        return [position, position+len(query_seq)]
    else:
        print('!!!Primer not found in the reference genome')
        return 0
    
def reverse_complement_sequence(seq):
    complement = {"A": "T", "T": "A", "C": "G", "G": "C"}
    reverse_seq = seq[::-1]
    return "".join(complement[base] for base in reverse_seq)

def check_for_snp(seq, priority, plength, forward, cut_off=1.3):
    if forward == True:
        ploc = find_sequence_location(seq)[0]
    else:    
        ploc = find_sequence_location(reverse_complement_sequence(seq))[0]
    snp_freq = priority[(priority['genome_pos']>= ploc) & (priority['genome_pos']<=ploc+plength)]['weight']
    print(snp_freq.max())
    if snp_freq.max() > cut_off:
        return True
    else:
        return False

In [2]:
full_data = pd.read_csv('snp_priority.csv')
# check_for_snp('CTCTTCGTCAGCTCCCACTC', full_data, 20, True, 1.5)

In [3]:
full_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 268368 entries, 0 to 268367
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   sample_id   268368 non-null  object 
 1   genome_pos  268368 non-null  int64  
 2   gene        268368 non-null  object 
 3   change      268368 non-null  object 
 4   freq        268368 non-null  float64
 5   type        268368 non-null  object 
 6   sublin      265459 non-null  object 
 7   drtype      268368 non-null  object 
 8   drugs       268368 non-null  object 
 9   weight      268368 non-null  float64
dtypes: float64(2), int64(1), object(7)
memory usage: 20.5+ MB


In [1]:
full_data

NameError: name 'full_data' is not defined

In [26]:
TmVar = {
    'DivalentSaltConc': 2,
    'MonovalentSaltConc': 10,
    'dNTPConc': 0.2,
    'OligoConc': 0.5
}


In [28]:
class TmVar:
	DivalentSaltConc = 2
	MonovalentSaltConc = 10
	dNTPConc = 0.2
	OligoConc = 0.5
 

In [32]:
TmVar['111'] = 111

TypeError: 'type' object does not support item assignment

# Susana design test

In [1]:
def calculate_gc_content(sequence):
    """
    Calculate the GC content of a DNA sequence.

    Parameters:
    - sequence: The DNA sequence (string)

    Returns:
    - float: The GC content as a percentage
    """
    
    # Check if the sequence is empty
    if len(sequence) == 0:
        return 0.0
    
    # Count the number of G's and C's in the sequence
    gc_count = sequence.count('G') + sequence.count('C')
    
    # Calculate the GC content
    gc_content = (gc_count / len(sequence)) * 100
    
    return gc_content



def calculate_tm_nearest_neighbor(sequence, na_concentration):
    if len(sequence) == 0:
        raise ValueError("Sequence length must be greater than 0")
    gc_count = sequence.count('G') + sequence.count('C')
    total_length = len(sequence)
    percent_gc = (gc_count / total_length) * 100
    tm = 81.5 + 16.6 * math.log10(na_concentration) + 0.41 * percent_gc - 675 / total_length
    return tm

def simplified_tm(seq):
    # Simplified melting temperature calculation based on base pair count
    return (seq.count('A') + seq.count('T')) * 2 + (seq.count('C') + seq.count('G')) * 4


In [25]:
max_gc = 0
min_gc = 100
max_dg = -100
min_dg = 100
max_tm = 0
min_tm = 100
max_tm_simple = 0
min_tm_simple = 100
for x in susana_design['forward'].tolist() + susana_design['reverse'].tolist(): 
    (dg, tm) = Thermo.simplified_tm(seq, complement_sequence(seq))
    if tm > max_tm:
        max_tm = tm
    elif tm < min_tm:
        min_tm = tm
    if dg['Gibbs'] > max_dg:
        max_dg = dg['Gibbs']
    elif dg['Gibbs'] < min_dg:
        min_dg = dg['Gibbs']
    # if calculate_tm_nearest_neighbor(x, 50) > max_tm:
    #     max_tm = calculate_tm_nearest_neighbor(x, 50)
    # elif calculate_tm_nearest_neighbor(x, 50) < min_tm:
    #     min_tm = calculate_tm_nearest_neighbor(x, 50)
    # if simplified_tm(x) > max_tm_simple:
    #     max_tm_simple = simplified_tm(x)
    # elif simplified_tm(x) < min_tm_simple:
    #     min_tm_simple = simplified_tm(x)
        
print('max_gc', max_gc)
print('min_gc', min_gc)
print('max_tm', max_tm)
print('min_tm', min_tm)
print('max_dg', max_dg)
print('min_dg', min_dg)
print('max_tm_simple', max_tm_simple)
print('min_tm_simple', min_tm_simple)

max_gc 0
min_gc 100
max_tm 63.024451980920674
min_tm 63.024451980920674
max_dg -24.099999999999998
min_dg -24.099999999999998
max_tm_simple 0
min_tm_simple 100


In [ ]:
from itertools import product

def calc_self_any(primer):
    max_score = 0
    for i in range(len(primer)):
        for j in range(i+1, len(primer)):
            score = 0
            for k, l in zip(primer[i:], primer[j:]):
                if k == l:
                    score += 1
            max_score = max(max_score, score)
    return max_score

def calc_self_end(primer):
    max_score = 0
    for i in range(len(primer)):
        score = 0
        for k, l in zip(primer[i:], reversed(primer[:len(primer) - i])):
            if k == l:
                score += 1
        max_score = max(max_score, score)
    return max_score

def calc_pair_compl_any(primer1, primer2):
    max_score = 0
    for i in range(len(primer1)):
        for j in range(len(primer2)):
            score = 0
            for k, l in zip(primer1[i:], primer2[j:]):
                if k == l:
                    score += 1
            max_score = max(max_score, score)
    return max_score

def calc_pair_compl_end(primer1, primer2):
    max_score = 0
    for i in range(len(primer1)):
        score = 0
        for k, l in zip(primer1[i:], reversed(primer2[:len(primer1) - i])):
            if k == l:
                score += 1
        max_score = max(max_score, score)
    return max_score

# Test the functions
primer1 = "ATCGATCG"
primer2 = "GCTAGCTA"

for x,y in zip(susana_design['forward'].tolist(), susana_design['reverse'].tolist()): 
    print("PRIMER_MAX_SELF_ANY for primer1:", calc_self_any(x))   
    print("PRIMER_MAX_SELF_END for x:", calc_self_end(x))   
    print("PRIMER_PAIR_MAX_COMPL_ANY:", calc_pair_compl_any(x, y))  
    print("PRIMER_PAIR_MAX_COMPL_END:", calc_pair_compl_end(x, y))  


In [13]:
ref_genome = 'MTB-h37rv_asm19595v2-eg18.fa'
def find_sequence_location(query_seq, fasta_file=ref_genome):
    ref_genome = SeqIO.read(fasta_file, "fasta")
    position = ref_genome.seq.find(query_seq)
    if position != -1:
        return [position, position+len(query_seq)]
    else:
        print('!!!Primer not found in the reference genome')
        return 0

# pLeft_coord = []
# pLeft_coord.append(find_sequence_location('CATCGCACGTCGTCTTTCCG', ref_genome)[0])

# find_sequence_location('CATCGCACGTCGTCTTTCCG')[0]
# find_sequence_location(complement_sequence('CATCGCACGTCGTCTTTCCG'))
#%%
def simplified_tm(seq):
    # Simplified melting temperature calculation based on base pair count
    return (seq.count('A') + seq.count('T')) * 2 + (seq.count('C') + seq.count('G')) * 4

def simplified_dg(seq):
    # Simplified Gibbs free energy calculation based on the nearest-neighbor model (not accurate)
    nn_params = {
        'AA': -1.0, 'TT': -1.0,
        'AT': -0.9, 'TA': -0.9,
        'CA': -1.7, 'TG': -1.7,
        'GT': -1.5, 'AC': -1.5,
        'CT': -1.6, 'AG': -1.6,
        'GA': -1.5, 'TC': -1.5,
        'CG': -2.8, 'GC': -2.3,
        'GG': -1.9, 'CC': -1.9
    }
    dg = 0
    for i in range(len(seq) - 1):
        dinucleotide = seq[i:i+2]
        dg += nn_params.get(dinucleotide, 0)
    return dg

def calculate_similarity(seq1, seq2):
    matches = sum(1 for a, b in zip(seq1, seq2) if a == b)
    return matches / len(seq1) * 100

def check_last_n_base_pairs(seq1, seq2, n):
    """
    Check if the last n base pairs of two sequences are the same.

    Parameters:
    - seq1: The first sequence (string)
    - seq2: The second sequence (string)
    - n: The number of base pairs to compare from the end (integer)

    Returns:
    - bool: True if the last n base pairs are the same, otherwise False
    """
    
    # Check if either sequence is shorter than n
    if len(seq1) < n or len(seq2) < n:
        return False
    
    # Get the last n base pairs from each sequence
    last_n_seq1 = seq1[-n:]
    last_n_seq2 = seq2[-n:]
    
    # Compare the last n base pairs
    return last_n_seq1 == last_n_seq2

# Example usage
# result = check_last_n_base_pairs("ATCGGA", "TTTGGA", 2)
# print("Are the last 2 base pairs the same?", result)  # Output should be True

# result = check_last_n_base_pairs("ATCGGA", "TTTGTA", 2)
# print("Are the last 2 base pairs the same?", result)  # Output should be False

#%%
def has_multiple_binding_sites(sequence, genome, similarity_threshold=90, min_tm=50, max_dg=-10):
    seq_len = len(sequence)
    genome_len = len(genome)
    count = 0
    n = 4
    for i in range(genome_len - seq_len + 1):
        within_tm_threshold, within_dg_threshold = False, False
        subseq = genome[i:i + seq_len]
        reverse_comp_subseq = reverse_complement_sequence(subseq)
        if calculate_similarity(subseq, sequence)>similarity_threshold:
            if check_last_n_base_pairs(subseq, sequence, n):
                # print('>>>Calculated_similarity')
                within_tm_threshold, within_dg_threshold = False, False
                tm = simplified_tm(subseq)
                dg = simplified_dg(subseq)
                
                within_tm_threshold = tm >= min_tm
                within_dg_threshold = dg <= max_dg
        elif calculate_similarity(reverse_comp_subseq, sequence)>similarity_threshold:
            if check_last_n_base_pairs(reverse_comp_subseq, sequence, n):
                # print('>>>Calculated_similarity')
                within_tm_threshold, within_dg_threshold = False, False
                tm = simplified_tm(reverse_comp_subseq)
                dg = simplified_dg(reverse_comp_subseq)
                
                within_tm_threshold = tm >= min_tm
                within_dg_threshold = dg <= max_dg
            
        if within_tm_threshold & within_dg_threshold:
            # print(genome[i:i + seq_len])
            # print('>>>within_tm_threshold & within_dg_threshold')
            count += 1
            if count > 1:
                print(max(calculate_similarity(subseq, sequence), calculate_similarity(reverse_comp_subseq, sequence)))
                return True  # Early exit if more than one binding site is found
    print(max(calculate_similarity(subseq, sequence), calculate_similarity(reverse_comp_subseq, sequence)))
    return False  # Return False if only one or no binding sites are found


In [11]:
def complement_sequence(dna_sequence):
    trans = str.maketrans('ATCG', 'TAGC')
    return dna_sequence.upper().translate(trans)

# print(complement_sequence("ATGCGTA"))
# Example usage:
# dna_sequence = "ATGCGTA"
# complement_sequence = complement_sequence(dna_sequence)
# print(complement_sequence)  # Output: TACGCAT
#%%
def reverse_complement_sequence(seq):
    complement = {"A": "T", "T": "A", "C": "G", "G": "C"}
    reverse_seq = seq[::-1]
    return "".join(complement[base] for base in reverse_seq)

def genome_size(fasta_file):
    total_length = 0
    with open(fasta_file, 'r') as file:
        for line in file:
            if not line.startswith('>'):
                total_length += len(line.strip())
    return total_length

def extract_sequence_from_fasta(start_pos, end_pos, padding = 150, fasta_file= 'MTB-h37rv_asm19595v2-eg18.fa', sequence_id='Chromosome'):
    """
    Extracts a subsequence from a FASTA file based on the given sequence ID, start position, and end position.
    """
    # Iterate over the sequences in the FASTA file
    for record in SeqIO.parse(fasta_file, "fasta"):
        # Check if the current sequence ID matches the desired sequence ID
        
        if record.id == sequence_id:
            # Extract the subsequence based on the start and end positions
            subsequence = record.seq[start_pos-padding:end_pos+padding]
            return str(subsequence)  # Return the subsequence as a string
    # If the sequence ID is not found, return None
    return None
genome = extract_sequence_from_fasta(0, genome_size(ref_genome),0)



In [14]:
for x in susana_design['forward'].tolist() + susana_design['reverse'].tolist(): 
    print(has_multiple_binding_sites(x, genome, similarity_threshold=80, min_tm=50, max_dg=-10))


83.33333333333334
True
85.0
True
100.0
True
82.35294117647058
True
10.0
False
100.0
True
10.0
False
20.0
False
20.0
False
35.0
False
35.0
False
25.0
False
31.57894736842105
False
82.35294117647058
True
83.33333333333334
True
82.35294117647058
True
84.21052631578947
True
100.0
True
82.35294117647058
True
82.35294117647058
True
30.0
False
30.0
False
25.0
False
83.33333333333334
True
84.21052631578947
True
100.0
True
94.11764705882352
True
36.84210526315789
False
35.0
False
100.0
True
82.35294117647058
True
83.33333333333334
True
82.35294117647058
True
35.0
False
85.71428571428571
True
81.81818181818183
True
20.0
False
85.0
True
88.23529411764706
True
82.35294117647058
True
15.0
False
88.23529411764706
True
31.57894736842105
False
35.0
False
30.0
False
30.0
False
84.21052631578947
True
85.0
True
20.0
False
20.833333333333336
False
100.0
True
84.21052631578947
True
31.57894736842105
False
20.0
False
82.35294117647058
True
82.35294117647058
True
30.0
False
30.0
False
83.33333333333334
True


KeyboardInterrupt: 

In [ ]:
# Example usage
gc_content = calculate_gc_content("ATCGGCAT")


In [2]:
from Bio.Blast.Applications import NcbiblastnCommandline
from Bio import SeqIO
from Bio.Blast import NCBIXML
import subprocess

In [2]:
def complement_sequence(dna_sequence):
    trans = str.maketrans('ATCG', 'TAGC')
    return dna_sequence.upper().translate(trans)

# print(complement_sequence("ATGCGTA"))
# # Example usage:
# # dna_sequence = "ATGCGTA"
# # complement_sequence = complement_sequence(dna_sequence)
# # print(complement_sequence)  # Output: TACGCAT
# #%%
def reverse_complement_sequence(seq):
    complement = {"A": "T", "T": "A", "C": "G", "G": "C"}
    reverse_seq = seq[::-1]
    return "".join(complement[base] for base in reverse_seq)


TACGCAT


In [ ]:
def calculate_mismatches(seq1, seq2):
    return sum(1 for a, b in zip(seq1, seq2) if a != b)

def find_alignments(sequence, genome, target_site):
    alignments = []
    seq_len = len(sequence)
    genome_len = len(genome)
    
    for i in range(genome_len - seq_len + 1):
        subseq = genome[i:i+seq_len]
        mismatches = calculate_mismatches(sequence, subseq)
        binding_location = (i, i+seq_len)
        distance_from_target = min(abs(i - target_site), abs(i + seq_len - target_site))
        
        alignment_info = {
            'binding_location': binding_location,
            'mismatches': mismatches,
            'distance_from_target': distance_from_target
        }
        alignments.append(alignment_info)
    
    return alignments

# Usage
sequence = 'ATG'
genome = 'ATGCGTATGACGTAG'
target_site = 7  # Assume the target site is at position 7
alignments = find_alignments(sequence, genome, target_site)

for alignment in alignments:
    binding_location = alignment['binding_location']
    mismatches = alignment['mismatches']
    distance_from_target = alignment['distance_from_target']
    print(f'Binding location: {binding_location}, Mismatches: {mismatches}, Distance from target: {distance_from_target}')



In [ ]:
def calculate_mismatches(seq1, seq2):
    return sum(1 for a, b in zip(seq1, seq2) if a != b)

def find_alignments(sequence, genome, target_site):
    alignments = []
    seq_len = len(sequence)
    genome_len = len(genome)
    
    for i in range(genome_len - seq_len + 1):
        subseq = genome[i:i+seq_len]
        mismatches = calculate_mismatches(sequence, subseq)
        binding_location = (i, i+seq_len)
        distance_from_target = min(abs(i - target_site), abs(i + seq_len - target_site))
        
        alignment_info = {
            'binding_location': binding_location,
            'mismatches': mismatches,
            'distance_from_target': distance_from_target
        }
        alignments.append(alignment_info)
    
    return alignments

# Usage
sequence = 'ATG'
genome = 'ATGCGTATGACGTAG'
target_site = 7  # Assume the target site is at position 7
alignments = find_alignments(sequence, genome, target_site)

for alignment in alignments:
    binding_location = alignment['binding_location']
    mismatches = alignment['mismatches']
    distance_from_target = alignment['distance_from_target']
    print(f'Binding location: {binding_location}, Mismatches: {mismatches}, Distance from target: {distance_from_target}')

In [76]:
ref_genome = '/mnt/storage10/lwang/Projects/Amplicone_design_tool/model/MTB-h37rv_asm19595v2-eg18.fa'

def genome_size(fasta_file):
    total_length = 0
    with open(fasta_file, 'r') as file:
        for line in file:
            if not line.startswith('>'):
                total_length += len(line.strip())
    return total_length

def extract_sequence_from_fasta(start_pos, end_pos, padding = 150, fasta_file= 'MTB-h37rv_asm19595v2-eg18.fa', sequence_id='Chromosome'):
    """
    Extracts a subsequence from a FASTA file based on the given sequence ID, start position, and end position.
    """
    # Iterate over the sequences in the FASTA file
    for record in SeqIO.parse(fasta_file, "fasta"):
        # Check if the current sequence ID matches the desired sequence ID
        
        if record.id == sequence_id:
            # Extract the subsequence based on the start and end positions
            subsequence = record.seq[start_pos-padding:end_pos+padding]
            return str(subsequence)  # Return the subsequence as a string
    # If the sequence ID is not found, return None
    return None

extract_sequence_from_fasta(0, genome_size(ref_genome))

'CGAGCCGTTGCCGGTAGGTTGCGGCTGGTTATCGACGGTACTGTCCACATTTGTGGATAGCCATGTGGACAGTTCACCTGCCCACAACAACGGTTGTAGCTCGACCCGGAACCAAGACCCGGAACTAACGAGAACCAGGGAGATACGTCG'

In [74]:
genome_size(ref_genome)

4411532

In [78]:
genome_size(ref_genome)

4411532

In [80]:
import primer_selection

TACGCAT


In [ ]:
primer_selection.extract_sequence_from_fasta(0, genome_size(ref_genome),0)

In [83]:
def simplified_tm(seq):
    # Simplified melting temperature calculation based on base pair count
    return (seq.count('A') + seq.count('T')) * 2 + (seq.count('C') + seq.count('G')) * 4

def simplified_dg(seq):
    nn_params = {
        'AA': -1.0, 'TT': -1.0,
        'AT': -0.9, 'TA': -0.9,
        'CA': -1.7, 'TG': -1.7,
        'GT': -1.5, 'AC': -1.5,
        'CT': -1.6, 'AG': -1.6,
        'GA': -1.5, 'TC': -1.5,
        'CG': -2.8, 'GC': -2.3,
        'GG': -1.9, 'CC': -1.9
    }
    dg = 0
    for i in range(len(seq) - 1):
        dinucleotide = seq[i:i+2]
        dg += nn_params.get(dinucleotide, 0)  # Adds 0 if dinucleotide not found
    return dg


def is_good_primer_candidate(primer, genome, target_site, max_mismatches, binding_location_range, max_distance_from_target, min_tm, max_dg):
    primer_len = len(primer)
    genome_len = len(genome)
    
    for i in range(genome_len - primer_len + 1):
        subseq = genome[i:i+primer_len]
        mismatches = calculate_mismatches(primer, subseq)
        distance_from_target = min(abs(i - target_site), abs(i + primer_len - target_site))
        
        # Check if the alignment meets the thresholds
        within_mismatch_threshold = mismatches <= max_mismatches
        within_binding_location_range = binding_location_range[0] <= i <= binding_location_range[1]
        within_distance_threshold = distance_from_target <= max_distance_from_target
        
        # Evaluate binding strength
        tm = simplified_tm(subseq)
        dg = simplified_dg(subseq)
        strong_binding = tm >= min_tm and dg <= max_dg
        
        # If a strong alternative binding site is found, return False
        summary = within_mismatch_threshold + within_binding_location_range + within_distance_threshold + strong_binding
        # if within_mismatch_threshold and within_binding_location_range and within_distance_threshold and strong_binding:
        if summary > 3:
            return False
    return True  # No strong alternative binding sites found

# Usage
primer = 'CGAACTCGAGGCTGCCTACT'
genome = primer_selection.extract_sequence_from_fasta(0, genome_size(ref_genome),0)
target_site = 7
max_mismatches = 2
binding_location_range = (0, 15)
max_distance_from_target = 50
min_tm = 50  # Example threshold for minimum melting temperature
max_dg = -10  # Example threshold for maximum Gibbs free energy

is_good_candidate = is_good_primer_candidate(primer, genome, target_site, max_mismatches, binding_location_range, max_distance_from_target, min_tm, max_dg)
print(is_good_candidate)


True


In [ ]:
    for i in range(genome_len - seq_len + 1):
        subseq = genome[i:i + seq_len]
        tm = simplified_tm(sequence)
        dg = simplified_dg(sequence)

        within_similarity_threshold = calculate_similarity(sequence, subseq, similarity_threshold)
        within_tm_threshold = tm >= min_tm
        within_dg_threshold = dg <= max_dg

        if within_similarity_threshold and within_tm_threshold and within_dg_threshold:
            count += 1
            if count > 1:
                return True  # Early exit if more than one binding site is found

    return False  # Return False if only one or no binding sites are found

## Below is the code used from testing alternative binding

In [140]:
def simplified_tm(seq):
    # Simplified melting temperature calculation based on base pair count
    return (seq.count('A') + seq.count('T')) * 2 + (seq.count('C') + seq.count('G')) * 4

def simplified_dg(seq):
    # Simplified Gibbs free energy calculation based on the nearest-neighbor model (not accurate)
    nn_params = {
        'AA': -1.0, 'TT': -1.0,
        'AT': -0.9, 'TA': -0.9,
        'CA': -1.7, 'TG': -1.7,
        'GT': -1.5, 'AC': -1.5,
        'CT': -1.6, 'AG': -1.6,
        'GA': -1.5, 'TC': -1.5,
        'CG': -2.8, 'GC': -2.3,
        'GG': -1.9, 'CC': -1.9
    }
    dg = 0
    for i in range(len(seq) - 1):
        dinucleotide = seq[i:i+2]
        dg += nn_params.get(dinucleotide, 0)
    return dg

def calculate_similarity(seq1, seq2):
    matches = sum(1 for a, b in zip(seq1, seq2) if a == b)
    return matches / len(seq1) * 100

def has_multiple_binding_sites(sequence, genome, similarity_threshold, min_tm=60, max_dg=-10):
    seq_len = len(sequence)
    genome_len = len(genome)
    count = 0

    for i in range(genome_len - seq_len + 1):
        subseq = genome[i:i + seq_len]
        if calculate_similarity(subseq, sequence)>similarity_threshold:
            tm = simplified_tm(subseq)
            dg = simplified_dg(subseq)
            
            within_tm_threshold = tm >= min_tm
            within_dg_threshold = dg <= max_dg

            if within_tm_threshold and within_dg_threshold:
                print(genome[i:i + seq_len])
                count += 1
                if count > 1:
                    return True  # Early exit if more than one binding site is found

    return False  # Return False if only one or no binding sites are found

# Usage:
primer = 'CGAACTCGAGGCTGCCTACT'
# primer = 'GCTCGTCCATGTCCCACCAT'
genome = primer_selection.extract_sequence_from_fasta(0, genome_size(ref_genome),0)
result = has_multiple_binding_sites(primer, genome, 81)
print(result)  # Output: True or False


CGAACTCGAGGCTGCCTACT
CGGACTCGAGGGTGCCTTCT
True


In [132]:
def has_multiple_binding_sites(sequence, genome):
    seq_len = len(sequence)
    genome_len = len(genome)
    count = 0

    for i in range(genome_len - seq_len + 1):
        subseq = genome[i:i + seq_len]
        if subseq == sequence:
            print(genome[i:i + seq_len])
            count += 1
            if count > 1:
                return True  # Early exit if more than one binding site is found

    return False  # Return False if only one or no binding sites are found

# Usage:
primer = 'CGAACTCGAGGCTGCCTACT'
# primer = 'GCTCGTCCATGTCCCACCAT'
genome = primer_selection.extract_sequence_from_fasta(0, genome_size(ref_genome),0)
result = has_multiple_binding_sites(primer, genome)
print(result)  # Output: True or False


CGAACTCGAGGCTGCCTACT
CGAACTCGAGGCTGCCTACT
True


In [103]:
def calculate_similarity(seq1, seq2):
    matches = sum(1 for a, b in zip(seq1, seq2) if a == b)
    return matches / len(seq1) * 100

def has_multiple_binding_sites(sequence, genome, similarity_threshold):
    seq_len = len(sequence)
    genome_len = len(genome)
    count = 0

    for i in range(genome_len - seq_len + 1):
        subseq = genome[i:i + seq_len]
        similarity = calculate_similarity(sequence, subseq)
        if similarity >= similarity_threshold:
            count += 1
            if count > 1:
                return True  # Early exit if more than one binding site is found

    return False  # Return False if only one or no binding sites are found

# Usage:
# primer = 'CGAACTCGAGGCTGCCTACT'
primer = 'GCTCGTCCATGTCCCACCAT'
genome = primer_selection.extract_sequence_from_fasta(0, genome_size(ref_genome),0)
similarity_threshold = 91  # For example, 90% similarity
result = has_multiple_binding_sites(primer, genome, similarity_threshold)
print(result)  # Output: True or False


False


In [104]:
def simplified_tm(seq):
    # Simplified melting temperature calculation based on base pair count
    return (seq.count('A') + seq.count('T')) * 2 + (seq.count('C') + seq.count('G')) * 4

def simplified_dg(seq):
    # Simplified Gibbs free energy calculation based on the nearest-neighbor model (not accurate)
    nn_params = {
        'AA': -1.0, 'TT': -1.0,
        'AT': -0.9, 'TA': -0.9,
        'CA': -1.7, 'TG': -1.7,
        'GT': -1.5, 'AC': -1.5,
        'CT': -1.6, 'AG': -1.6,
        'GA': -1.5, 'TC': -1.5,
        'CG': -2.8, 'GC': -2.3,
        'GG': -1.9, 'CC': -1.9
    }
    dg = 0
    for i in range(len(seq) - 1):
        dinucleotide = seq[i:i+2]
        dg += nn_params.get(dinucleotide, 0)
    return dg

def has_multiple_binding_sites(sequence, genome, similarity_threshold, min_tm, max_dg):
    seq_len = len(sequence)
    genome_len = len(genome)
    count = 0

    for i in range(genome_len - seq_len + 1):
        subseq = genome[i:i + seq_len]
        similarity = calculate_similarity(sequence, subseq)
        tm = simplified_tm(subseq)
        dg = simplified_dg(subseq)

        # Check if the alignment meets the thresholds
        within_similarity_threshold = similarity >= similarity_threshold
        within_tm_threshold = tm >= min_tm
        within_dg_threshold = dg <= max_dg

        if within_similarity_threshold and within_tm_threshold and within_dg_threshold:
            count += 1
            if count > 1:
                return True  # Early exit if more than one binding site is found

    return False  # Return False if only one or no binding sites are found

# Usage:
primer = 'GCTCGTCCATGTCCCACCAT'
genome = primer_selection.extract_sequence_from_fasta(0, genome_size(ref_genome),0)
similarity_threshold = 70  # For example, 90% similarity
min_tm = 40  # Example threshold for minimum melting temperature
max_dg = -10  # Example threshold for maximum Gibbs free energy

result = has_multiple_binding_sites(sequence, genome, similarity_threshold, min_tm, max_dg)
print(result)  # Output: True or False


False


In [130]:
def calculate_similarity(seq1, seq2, similarity_threshold):
    required_matches = len(seq1) * similarity_threshold / 100
    matches = 0
    for a, b in zip(seq1, seq2):
        if a == b:
            matches += 1
        if matches >= required_matches:
            return True
    return False    

def simplified_tm(seq):
    return (seq.count('A') + seq.count('T')) * 2 + (seq.count('C') + seq.count('G')) * 4

def simplified_dg(seq):
    nn_params = {
        'AA': -1.0, 'TT': -1.0,
        'AT': -0.9, 'TA': -0.9,
        'CA': -1.7, 'TG': -1.7,
        'GT': -1.5, 'AC': -1.5,
        'CT': -1.6, 'AG': -1.6,
        'GA': -1.5, 'TC': -1.5,
        'CG': -2.8, 'GC': -2.3,
        'GG': -1.9, 'CC': -1.9
    }
    dg = 0
    for i in range(len(seq) - 1):
        dinucleotide = seq[i:i+2]
        dg += nn_params.get(dinucleotide, 0)
    return dg

def precompute_dinucleotide_params(genome):
    unique_dinucleotides = set(genome[i:i+2] for i in range(len(genome) - 1))
    tm_params = {dinuc: simplified_tm(dinuc) for dinuc in unique_dinucleotides}
    dg_params = {dinuc: simplified_dg(dinuc) for dinuc in unique_dinucleotides}
    return tm_params, dg_params

tm_params, dg_params = precompute_dinucleotide_params(genome)

def has_multiple_binding_sites(sequence, genome, similarity_threshold, min_tm, max_dg):
    seq_len = len(sequence)
    genome_len = len(genome)
    count = 0
    for i in range(genome_len - seq_len + 1):
        subseq = genome[i:i + seq_len]
        tm = simplified_tm(sequence)
        dg = simplified_dg(sequence)

        within_similarity_threshold = calculate_similarity(sequence, subseq, similarity_threshold)
        within_tm_threshold = tm >= min_tm
        within_dg_threshold = dg <= max_dg

        if within_similarity_threshold and within_tm_threshold and within_dg_threshold:
            count += 1
            if count > 1:
                return True  # Early exit if more than one binding site is found

    return False  # Return False if only one or no binding sites are found

# Usage:
primer = 'GCTCGTCCATGTCCCACCAT'
genome = primer_selection.extract_sequence_from_fasta(0, genome_size(ref_genome),0)
similarity_threshold = 70  # For example, 90% similarity
min_tm = 40  # Example threshold for minimum melting temperature
max_dg = -10  # Example threshold for maximum Gibbs free energy

result = not has_multiple_binding_sites(sequence, genome, similarity_threshold, min_tm, max_dg)
print(result)  # Output: True or False

True


In [108]:
tm_params, dg_params = precompute_dinucleotide_params(genome)


In [118]:
tm = sum(tm_params[genome[i:i+2]] for i in range(len(primer) - 1))


In [119]:
tm

122

In [116]:
tm_params

{'CA': 6,
 'AC': 6,
 'TG': 6,
 'GG': 8,
 'TT': 4,
 'CT': 6,
 'CC': 8,
 'GA': 6,
 'AT': 4,
 'AA': 4,
 'GC': 8,
 'GT': 6,
 'AG': 6,
 'CG': 8,
 'TC': 6,
 'TA': 4}

In [4]:
def gc_content(sequence):
    if len(sequence) == 0:
        raise ValueError("Sequence length must be greater than 0")
    gc_count = sequence.count('G') + sequence.count('C')
    total_length = len(sequence)
    gc_content_percentage = (gc_count / total_length) * 100
    return gc_content_percentage

# Example usage:
sequence = 'GCTCGTCCATGTCCCACCAT'
gc_percentage = gc_content(sequence)
print(f'GC Content: {gc_percentage:.2f}%')


GC Content: 60.00%


In [3]:
import math

def calculate_tm_nearest_neighbor(sequence, na_concentration):
    if len(sequence) == 0:
        raise ValueError("Sequence length must be greater than 0")
    gc_count = sequence.count('G') + sequence.count('C')
    total_length = len(sequence)
    percent_gc = (gc_count / total_length) * 100
    tm = 81.5 + 16.6 * math.log10(na_concentration) + 0.41 * percent_gc - 675 / total_length
    return tm

# Example usage:
sequence = 'GCTCGTCCATGTCCCACCAT'
na_concentration = 0.05  # Example sodium ion concentration in mol/L
tm = calculate_tm_nearest_neighbor(sequence, na_concentration)
print(f'Melting Temperature (Tm): {tm:.2f}°C')


Melting Temperature (Tm): 50.75°C


In [5]:
def calculate_tm(sequence):
    if len(sequence) == 0:
        raise ValueError("Sequence length must be greater than 0")
    gc_count = sequence.count('G') + sequence.count('C')
    at_count = sequence.count('A') + sequence.count('T')
    tm = (gc_count * 4) + (at_count * 2)
    return tm

# Example usage:
sequence = 'GCTCGTCCATGTCCCACCAT'
tm = calculate_tm(sequence)
print(f'Melting Temperature (Tm): {tm}°C')


Melting Temperature (Tm): 64°C


In [2]:
spol = pd.read_csv('/mnt/storage10/lwang/Projects/Amplicone_design_tool/model/raw.githubusercontent.com_GaryNapier_spolpred_master_data_spoligotypes.csv')

In [4]:
spol.columns

Index(['id', 'pct_reads_mapped', 'num_reads_mapped', 'median_coverage',
       'main_lineage', 'sublineage', 'octal_spoligotype', 'binary_spoligotype',
       'spacer1', 'spacer2', 'spacer3', 'spacer4', 'spacer5', 'spacer6',
       'spacer7', 'spacer8', 'spacer9', 'spacer10', 'spacer11', 'spacer12',
       'spacer13', 'spacer14', 'spacer15', 'spacer16', 'spacer17', 'spacer18',
       'spacer19', 'spacer20', 'spacer21', 'spacer22', 'spacer23', 'spacer24',
       'spacer25', 'spacer26', 'spacer27', 'spacer28', 'spacer29', 'spacer30',
       'spacer31', 'spacer32', 'spacer33', 'spacer34', 'spacer35', 'spacer36',
       'spacer37', 'spacer38', 'spacer39', 'spacer40', 'spacer41', 'spacer42',
       'spacer43'],
      dtype='object')

# database check

In [3]:
# qwer = pd.read_csv('/mnt/storage7/jody/philippines/projects/all_philippines/tbprofiler_new/variants.txt')
s7_variants = pd.read_csv('/mnt/storage10/lwang/Projects/Amplicone_design_tool/model/variants.txt')

In [43]:
full_data = pd.read_csv('/mnt/storage10/jody/projects/variant_dump/variants.csv')


In [15]:
tb_drug_resistance_genes = {
    'gyrB': ['Levofloxacin'],
    'gyrA': ['Levofloxacin'],
    'mshA': ['Isoniazid'],
    'rpoB': ['Rifampicin'],
    'rpoC': ['Rifampicin'],
    'rpsL': ['Streptomycin'],
    'embR': ['Ethambutol'],
    'rrs': ['Kanamycin', 'Capreomycin', 'Amikacin', 'Streptomycin'],
    'fabG1': ['Isoniazid'],
    'inhA': ['Isoniazid'],
    'rpsA': ['Pyrazinamide'],
    'tlyA': ['Capreomycin'],
    'ndh': ['Isoniazid'],
    'katG': ['Isoniazid'],
    'pncA': ['Pyrazinamide'],
    'kasA': ['Isoniazid'],
    'eis': ['Kanamycin', 'Amikacin'],
    'ahpC': ['Isoniazid'],
    'rpoA': ['Rifampicin'],
    'panD': ['Pyrazinamide'],
    'embC': ['Ethambutol'],
    'embA': ['Ethambutol'],
    'embB': ['Ethambutol'],
    'ubiA': ['Ethambutol'],
    'gid': ['Streptomycin']
}
full_data['weight'] = full_data['freq']

full_data.loc[full_data['gene'].isin(tb_drug_resistance_genes.keys()), 'weight'] += 0.5


In [36]:
gene_names = [
    'PPE35', 'Rv1258c', 'Rv1979c', 'Rv2752c', 'Rv3083', 'Rv3236c', 'aftB', 'ahpC', 'ald', 'alr', 
    'atpE', 'ccsA', 'clpC1', 'ddn', 'eis', 'embA', 'embB', 'embC', 'embR', 'ethA', 'ethR', 'fabG1', 
    'fbiA', 'fbiB', 'fbiC', 'fbiD', 'fgd1', 'folC', 'fprA', 'gid', 'gyrA', 'gyrB', 'inhA', 'kasA', 
    'katG', 'mmpL5', 'mmpR5', 'mmpS5', 'mshA', 'ndh', 'panD', 'pepQ', 'pncA', 'ribD', 'rplC', 'rpoA', 
    'rpoB', 'rpoC', 'rpsA', 'rpsL', 'rrl', 'rrs', 'thyA', 'thyX', 'tlyA', 'ubiA', 'whiB6', 'whiB7'
]

In [ ]:
for x, n, g in zip(full_data.groupby('gene')['genome_pos'].max(), full_data.groupby('gene')['genome_pos'].min(), gene_names):
    gene_range = full_data[full_data['gene']==g]
    covered = gene_range[(gene_range['genome_pos']>=n)&(gene_range['genome_pos']<=x)]
    print('==================================')
    print(x, n)
    print(gene_range['genome_pos'].max(), gene_range['genome_pos'].min())
    print(g, len(covered), len(gene_range), len(covered)/len(gene_range))
    

In [ ]:
for x, n, g in zip(full_data.groupby('gene')['genome_pos'].max(), full_data.groupby('gene')['genome_pos'].min(), gene_names):
    gene_range = s7_variants[s7_variants['gene']==g]
    covered = gene_range[(gene_range['genome_pos']>=n)&(gene_range['genome_pos']<=x)]
    print('==================================')
    print(x, n)
    print(gene_range['genome_pos'].max(), gene_range['genome_pos'].min())
    print(g, len(covered), len(gene_range), len(covered)/len(gene_range))
    

In [21]:
overlap = set(s7_variants['sample_id'].unique().tolist()).intersection(set(full_data['sample_id'].unique().tolist()))


In [ ]:
full_data[full_data['change']=='p.Ala403Ser']

In [7]:
full_data[full_data['change']=='p.Ala403Ser']

,sample_id,genome_pos,gene,change,freq,type,sublin,drtype,drugs
0,ERR9787047,6446,gyrB,p.Ala403Ser,1.000000,missense_variant,lineage6.1.1,Sensitive,NaN
141,SRR8902561,6446,gyrB,p.Ala403Ser,1.000000,missense_variant,La1.8.1,Other,NaN
2273,ERR2815611,6446,gyrB,p.Ala403Ser,0.985075,missense_variant,La1.1,Sensitive,NaN
2336,ERR3439434,6446,gyrB,p.Ala403Ser,1.000000,missense_variant,La1.8.1,Other,NaN
3706,SRR14782548,6446,gyrB,p.Ala403Ser,0.976190,missense_variant,La1.2.BCG,Other,NaN
...,...,...,...,...,...,...,...,...,...
7414917,ERR2512732,6446,gyrB,p.Ala403Ser,1.000000,missense_variant,La1.8.1,Other,NaN
7415030,ERR192047,6446,gyrB,p.Ala403Ser,1.000000,missense_variant,La1.8.1,Other,NaN
7415606,ERR4192560,6446,gyrB,p.Ala403Ser,1.000000,missense_variant,lineage6.1.1,Sensitive,NaN
7415792,SRR1735567,6446,gyrB,p.Ala403Ser,1.000000,missense_variant,La1.2.BCG,Other,NaN


In [5]:
full_data.shape

(7420996, 9)

In [ ]:
full_data

In [19]:
import random

def generate_random_gene_seq(length):
    bases = ['A', 'T', 'C', 'G']
    return ''.join(random.choice(bases) for _ in range(length))

random_gene_seq = generate_random_gene_seq(300)
# print(f"Random Gene Sequence of Length 100: {random_gene_seq}")


In [23]:
random_gene_seq

'CGTCTGTACATGCCGCTCTATTACAATTATTCCACAAGGGTCAAGGGGCCGCTGCGCTCGGAACGCTCGTGATAGCGCTATCTCAACGCTATTACGGTAGATTCGACTTAAACACATACCTAAGCGCAAGGTAGCACGGCGGGCGTCTTTTAAACTTGGGCATCGCGACCACCGGCCAATTACGGTTTAGCTATCGTCTTACTTAGGCTATAGAGCGCTTTGCCAGAACAAAGGAAAGCGTAAGGATAACCACGGTTCGATGTTGATGGGCGACGACAGGTAGCTCGATCCTCCACTCCG'

In [ ]:
import primer3

# Define sequence arguments
seq_args = {
    'SEQUENCE_ID': 'example',
    'SEQUENCE_TEMPLATE': random_gene_seq,
    'SEQUENCE_INCLUDED_REGION': [15, len(random_gene_seq) - 15]
}

# Define global arguments
global_args = {
    'PRIMER_TASK': 'generic',
    'PRIMER_PICK_LEFT_PRIMER': 1,
    'PRIMER_PICK_INTERNAL_OLIGO': 1,
    'PRIMER_PICK_RIGHT_PRIMER': 1,
    'PRIMER_OPT_SIZE': 20,
    'PRIMER_MIN_SIZE': 18,
    'PRIMER_MAX_SIZE': 25,
    'PRIMER_NUM_RETURN': 5,
    'PRIMER_MIN_TM': 57.0,
    'PRIMER_MAX_TM': 63.0,
    'PRIMER_MIN_GC': 20.0,
    'PRIMER_MAX_GC': 80.0,
    'PRIMER_MAX_POLY_X': 100,
    'PRIMER_INTERNAL_MAX_POLY_X': 100,
    'PRIMER_SALT_MONOVALENT': 50.0,
    'PRIMER_DNA_CONC': 50.0,
    'PRIMER_MAX_NS_ACCEPTED': 0,
    'PRIMER_MAX_SELF_ANY': 12,
    'PRIMER_MAX_SELF_END': 8,
    'PRIMER_PAIR_MAX_COMPL_ANY': 12,
    'PRIMER_PAIR_MAX_COMPL_END': 8
}

# Run the primer design
designs = primer3.design_primers(seq_args, global_args)
designs = bindings.design_primers(seq_args, global_args)

# Output results
for key, value in designs.items():
    print(f"{key}: {value}")


In [12]:
# Print to validate
print("Start of SEQUENCE_INCLUDED_REGION: ", seq_args['SEQUENCE_INCLUDED_REGION'][0])
print("Length of SEQUENCE_INCLUDED_REGION: ", seq_args['SEQUENCE_INCLUDED_REGION'][1])
print("Total length of SEQUENCE_TEMPLATE: ", len(seq_args['SEQUENCE_TEMPLATE']))

# Validate bounds
if seq_args['SEQUENCE_INCLUDED_REGION'][0] + seq_args['SEQUENCE_INCLUDED_REGION'][1] > len(seq_args['SEQUENCE_TEMPLATE']):
    print("Error: SEQUENCE_INCLUDED_REGION is out of bounds")


Start of SEQUENCE_INCLUDED_REGION:  1
Length of SEQUENCE_INCLUDED_REGION:  70
Total length of SEQUENCE_TEMPLATE:  71


In [3]:
seq = 'AGTGCTTTCGCACGTCGTACCCCTCGGGTAGCTGGCGCTGGATCAGGCTCAGGAACATCTTCCGCATGCGCCGTGGCCACTTCTGGCAGGCGCTGTACACGGCCGCCTGGCGCAGCACGTTCTTCCACCGTACCGCGGTGTAGGCCATGGTCTCCGGCAGCCAGCGGTTGAGCTTCTCGGCGATGCCGTCCCGGTCTGGCTGCGACACGATGTAGGTGGGTGAGCGCTGCAGCATCGTGACGTGCTTGGCGCCCGAGTCCGCCAGCGCCGGCACGAGCGTGACCGCCGTTGCGCCACTGCCGATCACGACGATGTTCTTAGCGTCGTAGTCGAGGTCCTCGGGCCAGTGCTGCGGATGGATGATCGGCCCGACGAAATCCTCCGAGCCGGCGAATCTCGGCGAGTAGCCCTCGTCGTAGTTGTAGTAGCCGCTGCACAGAAAGAGGAATTCGCAGGTGAGGGCGCTGAGCGTGCCGTGGCTTTGGATGTGAACGGTCCAGCGGTTTTCCGCGGTCGACCAATCGGCACTGATCACCTTGTGGTGGAACCGGATATGCCTGTCGATTCCATACATGGCCGCGGTGCTCTTGACGTACTCGAGGATGGGCTTGCCGTCGGCGATCGCCTGCCGTCCGGTCCAGGGACGGAATCGGAAACCTAGCGTGTACATGTCGGAGTCGGAGCGAATTCCGGGATAACGGAACAAATCCCAGGTGCCGCCCATGGATTCCCGCTTTTCCAGGATGGCGTAGCTCTTGGTCGGGCAACGGTCCTGCAGGTGCCAGGCCGCGCTGACACCGGAGATTCCAGCGCCCACGATGACAACGTCGAGGTGCTCGGTCATGGATCCACGCTATCAACGTAATGTCGAGGCCGTCAACGAGATGTCGACACTATCGACACGTAGTAAGCTGCCAGGGTGACCACCTCCGCGGCCAGTCAGGCTTCGCTGCCTAGGGGCCGGCGCACCGCGCGGCCGTCCGGCGACGATCGTGAACTGGCGATCCTCGCCACCGCCGAGAACCTTCTCGAGGACCGTCCGCTGGCCGATATCTCGGTCGACGATCTGGCCAAGGGCGCCGGTATCTCGAGGCCGACGTTCTACTTCTATTTCCCATCCAAGGAAGCGGTGCTGCTGACCCTGCTGGACCGGGTGGTCAATCAAGCCGACATGGCCCTACAGACCCTTGCCGAGAATCCCGCCGACACCGACCGCGAGAACATGTGGCGCACCGGGATCAACGTGTTCTTCGAGACATTCGGGTCGCACAAGGCGGTAACCCGAGCCGGTCAGGCCGCCAGGGCAACCAGTGTCGAAGTCGCCGAACTGTGGTCGACGTTTATGCAGAAGTGGATCGCCTACACGGCCGCCGTGATCGACGCCGAACGCGACCGAGGCGCGGCGCCGCGCACCCTGCCGGCCCATGAACTGGCCACAGCGCTCAACCTGATGAACGAGCGGACGCTGTTCGCGTCATTCGCCGGCGAACAGCCCTCGGT'
len(seq)

1500

In [8]:
seq[205:230]

'CACGATGTAGGTGGGTGAGCGCTGC'

In [ ]:
ACTGCCGATCACGACGATGT

In [13]:
left_seq = 'AGTGCTTTCGCACGTCGTACCCCTCGGGTAGCTGGCGCTGGATCAGGCTCAGGAACATCTTCCGCATGCGCCGTGGCCACTTCTGGCAGGCGCTGTACACGGCCGCCTGGCGCAGCACGTTCTTCCACCGTACCGCGGTGTAGGCCATGGTCTCCGGCAGCCAGCGGTTGAGCTTCTCGGCGATGCCGTCCCGGTCTGGCTGCGACACGATGTAGGTGGGTGAGCGCTGCAGCATCGTGACGTGCTTGGCGCCCGAGTCCGCCAGCGCCGGCACGAGCGTGACCGCCGTTGCGCC'
len(left_seq)

295

In [10]:
left_seq

'AGTGCTTTCGCACGTCGTACCCCTCGGGTAGCTGGCGCTGGATCAGGCTCAGGAACATCTTCCGCATGCGCCGTGGCCACTTCTGGCAGGCGCTGTACACGGCCGCCTGGCGCAGCACGTTCTTCCACCGTACCGCGGTGTAGGCCATGGTCTCCGGCAGCCAGCGGTTGAGCTTCTCGGCGATGCCGTCCCGGTCTGGCTGCGACACGATGTAGGTGGGTGAGCGCTGCAGCATCGTGACGTGCTTGGCGCCCGAGTCCGCCAGCGCCGGCACGAGCGTGACCGCCGTTGCGCC'

In [12]:
primer_selection.extract_sequence_from_fasta(4327174, 4327174+20, padding=0)

'AACCGGATATGCCTGTCGAT'

In [15]:
primer_selection.extract_sequence_from_fasta(4327129, 4327129+20, padding=0)


'CGGTTTTCCGCGGTCGACCA'

In [14]:
4327129 + 295

4327424

In [36]:
from Bio import SeqIO

def find_sequence_location(fasta_file, query_sequence):
    for record in SeqIO.parse(fasta_file, "fasta"):
        sequence = str(record.seq)
        if query_sequence in sequence:
            start_index = sequence.index(query_sequence)
            end_index = start_index + len(query_sequence)
            return f"Sequence found in {record.id} from position {start_index} to {end_index}"
    return "Sequence not found in the FASTA file"

# Example usage
fasta_file = "/mnt/storage10/lwang/Projects/Amplicone_design_tool/MTB-h37rv_asm19595v2-eg18.fa"
query_sequence = "ATCATCAACGGGACCGAGCG"

result = find_sequence_location(fasta_file, query_sequence)
print(result)


Sequence found in Chromosome from position 760286 to 760306


In [40]:
fasta_file = "/mnt/storage10/lwang/Projects/Amplicone_design_tool/MTB-h37rv_asm19595v2-eg18.fa"
query_sequence = "AGTCCACCGACAAGACGCTG"

result = find_sequence_location(fasta_file, query_sequence)
print(result)

Sequence found in Chromosome from position 760365 to 760385


In [35]:
4328229-350

4327879

In [41]:
find_sequence_location(fasta_file, 'CCGGTTCAGGCTTCACCACA')

'Sequence found in Chromosome from position 13 to 33'

In [45]:
from Bio import SeqIO

def find_sequence_location(query_sequence, position, fasta_file):
    closest_start = float('inf')
    closest_end = float('inf')
    closest_record = None

    for record in SeqIO.parse(fasta_file, "fasta"):
        sequence = str(record.seq)
        index = sequence.find(query_sequence)
        
        while index != -1:
            start_index = index
            end_index = index + len(query_sequence)
            
            if abs(position - start_index) < abs(position - closest_start):
                closest_start = start_index
                closest_end = end_index
                closest_record = record.id

            index = sequence.find(query_sequence, index + 1)

    if closest_record is not None:
        return closest_start, closest_end
    else:
        print("Primer Sequence not found in FASTA file.")
        return 0,0

In [50]:
find_sequence_location('GCCTAAACCGGCGCTAAACC', 4326008, fasta_file='/mnt/storage10/lwang/Projects/Amplicone_design_tool/MTB-h37rv_asm19595v2-eg18.fa')


(4325973, 4325993)

In [28]:
primer_selection.extract_sequence_from_fasta(4326924, 4326944, padding=0)


'ACTGCCGATCACGACGATGT'

In [25]:
from Bio import SeqIO

def find_sequence_location(fasta_file, query_sequence, position):
    closest_start = float('inf')
    closest_end = float('inf')
    closest_record = None

    for record in SeqIO.parse(fasta_file, "fasta"):
        sequence = str(record.seq)
        index = sequence.find(query_sequence)
        
        while index != -1:
            start_index = index
            end_index = index + len(query_sequence)
            
            if abs(position - start_index) < abs(position - closest_start):
                closest_start = start_index
                closest_end = end_index
                closest_record = record.id

            index = sequence.find(query_sequence, index + 1)

    if closest_record is not None:
        return closest_start, closest_end
    else:
        return "Sequence not found in the FASTA file"

In [29]:
query_sequence ='ACTGCCGATCACGACGATGT'
# query_sequence = primer_selection.reverse_complement_sequence(query_sequence)
find_sequence_location(fasta_file, query_sequence, 4327932)[0]
# print(result)

4326924

In [ ]:
4327129 4327629

In [ ]:
4327129

In [20]:
[[4326879, 4327879],
 [2282776, 2283776],
 [2285958, 2286958],
 [4246734, 4247734],
 [4244193, 4245193],
 [2280899, 2281899],
 [2152912, 2153912],
 [2284954, 2285954],
 [2747141, 2748141],
 [1834840, 1835840],
 [2150990, 2151990],
 [2277174, 2278174],
 [3067958, 3068958],
 [2147188, 2148188],
 [2138550, 2139550],
 [2281712, 2282712],
 [2283527, 2284527],
 [1416371, 1417371],
 [3840393, 3841393],
 [3073502, 3074502],
 [2149884, 2150884],
 [2276296, 2277296],
 [2279554, 2280554],
 [2287239, 2288239],
 [2141412, 2142412],
 [2140159, 2141159],
 [2148199, 2149199],
 [2144518, 2145518],
 [3086947, 3087947],
 [2128861, 2129861]]

4326629